In [1]:
import numpy as np
import pandas as pd

In [2]:
import json
import matplotlib.cm as cm
import matplotlib.colors as colors

import xml

from geopy.geocoders import Nominatim


In [3]:
!conda install -c conda-forge geopy --yes

Solving environment: done

# All requested packages already installed.



In [4]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

# All requested packages already installed.



In [5]:
import folium

In [6]:
import requests
from pandas.io.json import json_normalize

from sklearn.cluster import KMeans

In [7]:
from bs4 import BeautifulSoup

### Scape from wikipedia

In [8]:
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(url,'lxml')
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xh6LeApAIDEAACUs1LEAAACJ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":935851093,"wgRevisionId":935851093,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario

### Creating Dataframe

In [9]:
table_post=soup.find('table')
fields=table_post.find_all('td')
postcode=[]
borough=[]
neighborhood=[]

In [10]:
for i in range(0,len(fields),3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighborhood.append(fields[i+2].text.strip())
    

In [11]:
df_pc=pd.DataFrame(data=[postcode,borough,neighborhood]).transpose()
df_pc.columns=['PostalCode', 'Borough', 'Neighborhood']
df_pc.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Remove 'Not assigned' Neighborhood

In [12]:
df_pc['Borough'].replace('Not assigned',np.nan,inplace=True)

In [13]:
df_pc.dropna(subset=['Borough'],inplace=True)

In [14]:
df_pc.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [15]:
df_pc1=df_pc.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_pc1.columns=['PostalCode', 'Borough', 'Neighborhood']
df_pc1

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [16]:
df_pc1['Neighborhood'].replace("Not assigned","Queen's Park",inplace=True)
df_pc1

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [17]:
df_pc1.shape

(103, 3)

In [18]:
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns=['PostalCode', 'Latitude', 'Longitude']

In [19]:
df_pos=pd.merge(df_pc1,df_geo,on=['PostalCode'],how='inner')
df_toronto=df_pos[['Borough','Neighborhood','PostalCode','Latitude','Longitude']].copy()
df_toronto.head()

,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [20]:
address='Toronto, Canada'

In [21]:
geolocator=Nominatim()
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print("The geocordinates of Toronto city are: {}, {}.".format(latitude,longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


The geocordinates of Toronto city are: 43.653963, -79.387207.


### Mapping Toronto city data

In [22]:
map_toronto=folium.Map(location=[latitude,longitude],zoom_start=10)


In [23]:
for lat,lng,borough,neighborhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighborhood']):
    label='{},{}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng], radius=3,popup=label,color='blue',fill=True,fill_color='#3199cc',fill_opacity=0.3,parse_html=False).add_to(map_toronto)

In [24]:
map_toronto

### Exploring the first neighborhood

In [71]:
df_toronto.loc[0,'Neighborhood']

'Rouge, Malvern'

### Fetching the location coordinates for the above location

In [72]:
neighborhood_lat=df_toronto.loc[0,'Latitude']
neighborhood_lng=df_toronto.loc[0,'Longitude']
neighborhood_neighborhoodname=df_toronto.loc[0,'Neighborhood']

print('Neighborhood {} has geo-coordinates latitude {} and longitude {} '.format(neighborhood_neighborhoodname,neighborhood_lat,neighborhood_lng))

Neighborhood Rouge, Malvern has geo-coordinates latitude 43.806686299999996 and longitude -79.19435340000001 


### Fetching top 100 venues within 800meter radius

In [73]:
CLIENT_ID='UVEWOYLYHMRELU5DLIW04GO0ZCWXDXYWBY3CNPZKNWFGNH1F'
CLIENT_SECRET='3DFA4GSX553U3O1CPZS5YIDLBNVUGNPBFV1OU0VWQXS04LJT'
VERSION='20180605'


In [82]:
LIMIT=100
radius=800

url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,neighborhood_lat,neighborhood_lng,radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=UVEWOYLYHMRELU5DLIW04GO0ZCWXDXYWBY3CNPZKNWFGNH1F&client_secret=3DFA4GSX553U3O1CPZS5YIDLBNVUGNPBFV1OU0VWQXS04LJT&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=800&limit=100'

In [83]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2251e00de0d9001b5eff7c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 14,
  'suggestedBounds': {'ne': {'lat': 43.8138863072, 'lng': -79.18439528916866},
   'sw': {'lat': 43.79948629279999, 'lng': -79.20431151083136}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs': [{'labe

### Extracting venue categories

In [84]:
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list) ==0:
        return None
    else:
        return categories_list[0]['name']

### Clean json and structure into a pandas dataframe

In [85]:
venues=results['response']['groups'][0]['items']


In [86]:
nearby_venues=json_normalize(venues)

In [87]:
filtered_Columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']

In [88]:
nearby_venues=nearby_venues.loc[:,filtered_Columns]

In [89]:
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type,axis=1)

In [90]:
nearby_venues_columns=[col.split(".")[-1] for col in nearby_venues.columns]

In [93]:
nearby_venues.head(20)

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Wendy's,Fast Food Restaurant,43.802008,-79.198080
2,Wendy's,Fast Food Restaurant,43.807448,-79.199056
3,Tim Hortons,Coffee Shop,43.802000,-79.198169
4,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
5,Harvey's,Fast Food Restaurant,43.800106,-79.198258
6,Lee Valley,Hobby Shop,43.803161,-79.199681
7,FASTSIGNS,Business Service,43.807882,-79.201968
8,Bus Stop: 85 & 116,Bus Station,43.802198,-79.199389
9,Tim Hortons / Esso,Coffee Shop,43.801863,-79.199296


In [92]:
print('The number of venues returned by foursquare API are:{}'.format(nearby_venues.shape[0]))

The number of venues returned by foursquare API are:14


In [98]:
map_toronto=folium.Map(location=[latitude,longitude],zoom_start=20)

In [103]:
for lat,lng,borough,neighborhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighborhood']):
    label='{},{}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng], radius=3,popup=label,color='purple',fill=True,fill_color='#3199cc',fill_opacity=0.3,parse_html=False).add_to(map_toronto)

In [104]:
map_toronto